In [1]:
import os
import pandas as pd
import numpy as np

In [2]:
os.listdir('./')

['.ipynb_checkpoints',
 'gender_submission.csv',
 'submission.csv',
 'test.csv',
 'titanic_analysis01.ipynb',
 'titanic_analysis02.ipynb',
 'titanic_analysis03.ipynb',
 'titanic_analysis04.ipynb',
 'titanic_analysis05.ipynb',
 'train.csv']

In [3]:
df = pd.read_csv('./train.csv')
df.head(10)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
5,6,0,3,"Moran, Mr. James",male,NaN,0,0,330877,8.4583,NaN,Q
6,7,0,1,"McCarthy, Mr. Timothy J",male,54.0,0,0,17463,51.8625,E46,S
7,8,0,3,"Palsson, Master. Gosta Leonard",male,2.0,3,1,349909,21.0750,NaN,S
8,9,1,3,"Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)",female,27.0,0,2,347742,11.1333,NaN,S
9,10,1,2,"Nasser, Mrs. Nicholas (Adele Achem)",female,14.0,1,0,237736,30.0708,NaN,C


In [4]:
df_cabin = df[["Pclass","Ticket","Fare","Embarked", "Cabin"]]
df_cabin = df_cabin.dropna(how="any").reset_index(drop=True)
df_cabin

,Pclass,Ticket,Fare,Embarked,Cabin
0,1,PC 17599,71.2833,C,C85
1,1,113803,53.1000,S,C123
2,1,17463,51.8625,S,E46
3,3,PP 9549,16.7000,S,G6
4,1,113783,26.5500,S,C103
...,...,...,...,...,...
197,1,11751,52.5542,S,D35
198,1,695,5.0000,S,B51 B53 B55
199,1,11767,83.1583,C,C50
200,1,112053,30.0000,S,B42


In [5]:
tempList = []
def filterOutlier(x) :
    for i in x :
        if(i.find(' ')>0 or len(i) == 1):
            tempList.append(np.nan)
        else:
            tempList.append(i)
    return tempList

In [6]:
df_cabin["valid"] = df_cabin[["Cabin"]].apply(lambda x: filterOutlier(x))

In [7]:
df_cabin = df_cabin.dropna(how = "any").reset_index(drop = True)
df_cabin

,Pclass,Ticket,Fare,Embarked,Cabin,valid
0,1,PC 17599,71.2833,C,C85,C85
1,1,113803,53.1000,S,C123,C123
2,1,17463,51.8625,S,E46,E46
3,3,PP 9549,16.7000,S,G6,G6
4,1,113783,26.5500,S,C103,C103
...,...,...,...,...,...,...
169,1,PC 17590,50.4958,S,A24,A24
170,1,11751,52.5542,S,D35,D35
171,1,11767,83.1583,C,C50,C50
172,1,112053,30.0000,S,B42,B42


In [8]:
df_cabin["Cabin"].unique()
list = []
for i in df_cabin["Cabin"]:
    list.append(i[0])
ser = pd.Series(list)
print(ser.unique())

['C' 'E' 'G' 'D' 'A' 'B' 'F']


In [9]:
tempList=[]
def seperateFloor(x):
    for i in x:
        if pd.isna(i):
            tempList.append(np.nan)
        elif i[0] == 'A':
            tempList.append(7)
        elif i[0] == 'B':
            tempList.append(6)
        elif i[0] == 'C':
            tempList.append(5)
        elif i[0] == 'D':
            tempList.append(4)
        elif i[0] == 'E':
            tempList.append(3)
        elif i[0] == 'F':
            tempList.append(2)
        elif i[0] == 'G':
            tempList.append(1)
        else:
            tempList.append(np.nan)
    return tempList

In [10]:
df_cabin["Floor"] = df_cabin[["Cabin"]].apply(lambda x: seperateFloor(x))
df_cabin

,Pclass,Ticket,Fare,Embarked,Cabin,valid,Floor
0,1,PC 17599,71.2833,C,C85,C85,5
1,1,113803,53.1000,S,C123,C123,5
2,1,17463,51.8625,S,E46,E46,3
3,3,PP 9549,16.7000,S,G6,G6,1
4,1,113783,26.5500,S,C103,C103,5
...,...,...,...,...,...,...,...
169,1,PC 17590,50.4958,S,A24,A24,7
170,1,11751,52.5542,S,D35,D35,4
171,1,11767,83.1583,C,C50,C50,5
172,1,112053,30.0000,S,B42,B42,6


In [11]:
from sklearn.linear_model import LogisticRegression

In [12]:
floor_model = LogisticRegression(max_iter=5000).fit(df_cabin[["Pclass", "Fare"]].join(pd.get_dummies(df_cabin[["Embarked"]])), df_cabin[["Floor"]])

C:\Users\30133\anaconda3\envs\myenv\lib\site-packages\sklearn\utils\validation.py:985: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\30133\anaconda3\envs\myenv\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [13]:
df_x = df[["PassengerId", "Pclass", "Name", "Sex", "Age", "SibSp", "Parch", "Ticket", "Fare", "Embarked", "Cabin"]]
df_y = df[["PassengerId", "Survived"]]
df_x

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Embarked,Cabin
0,1,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,S,NaN
1,2,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C,C85
2,3,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,S,NaN
3,4,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,S,C123
4,5,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,S,NaN
...,...,...,...,...,...,...,...,...,...,...,...
886,887,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,S,NaN
887,888,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,S,B42
888,889,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,S,NaN
889,890,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C,C148


In [14]:
df_x["Cabin"].unique()

array([nan, 'C85', 'C123', 'E46', 'G6', 'C103', 'D56', 'A6',
       'C23 C25 C27', 'B78', 'D33', 'B30', 'C52', 'B28', 'C83', 'F33',
       'F G73', 'E31', 'A5', 'D10 D12', 'D26', 'C110', 'B58 B60', 'E101',
       'F E69', 'D47', 'B86', 'F2', 'C2', 'E33', 'B19', 'A7', 'C49', 'F4',
       'A32', 'B4', 'B80', 'A31', 'D36', 'D15', 'C93', 'C78', 'D35',
       'C87', 'B77', 'E67', 'B94', 'C125', 'C99', 'C118', 'D7', 'A19',
       'B49', 'D', 'C22 C26', 'C106', 'C65', 'E36', 'C54',
       'B57 B59 B63 B66', 'C7', 'E34', 'C32', 'B18', 'C124', 'C91', 'E40',
       'T', 'C128', 'D37', 'B35', 'E50', 'C82', 'B96 B98', 'E10', 'E44',
       'A34', 'C104', 'C111', 'C92', 'E38', 'D21', 'E12', 'E63', 'A14',
       'B37', 'C30', 'D20', 'B79', 'E25', 'D46', 'B73', 'C95', 'B38',
       'B39', 'B22', 'C86', 'C70', 'A16', 'C101', 'C68', 'A10', 'E68',
       'B41', 'A20', 'D19', 'D50', 'D9', 'A23', 'B50', 'A26', 'D48',
       'E58', 'C126', 'B71', 'B51 B53 B55', 'D49', 'B5', 'B20', 'F G63',
       'C62 C64',

In [15]:
tempList=[]
def seperateFloor(x):
    for i in x:
        if pd.isna(i):
            tempList.append(np.nan)
        elif i[0] == 'A':
            tempList.append(7)
        elif i[0] == 'B':
            tempList.append(6)
        elif i[0] == 'C':
            tempList.append(5)
        elif i[0] == 'D':
            tempList.append(4)
        elif i[0] == 'E':
            tempList.append(3)
        elif i[0] == 'F':
            tempList.append(2)
        elif i[0] == 'G':
            tempList.append(1)
        else:
            tempList.append(np.nan)
    return tempList

In [16]:
df_x["Floor"] = df_x[["Cabin"]].apply(lambda x: seperateFloor(x))
df_x

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Embarked,Cabin,Floor
0,1,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,S,NaN,NaN
1,2,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C,C85,5.0
2,3,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,S,NaN,NaN
3,4,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,S,C123,5.0
4,5,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,S,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,S,NaN,NaN
887,888,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,S,B42,6.0
888,889,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,S,NaN,NaN
889,890,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C,C148,5.0


In [17]:
df_x["FloorPredict"] = floor_model.predict(df_x[["Pclass", "Fare"]].join(pd.get_dummies(df_x[["Embarked"]])))

In [18]:
df_x

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Embarked,Cabin,Floor,FloorPredict
0,1,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,S,NaN,NaN,1
1,2,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C,C85,5.0,5
2,3,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,S,NaN,NaN,1
3,4,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,S,C123,5.0,5
4,5,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,S,NaN,NaN,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,S,NaN,NaN,3
887,888,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,S,B42,6.0,3
888,889,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,S,NaN,NaN,3
889,890,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C,C148,5.0,7


In [19]:
for i in range(0,df_x.shape[0]):
    if (pd.isna(df_x["Floor"][i])):
        df_x["Floor"][i] = int(np.round(df_x["FloorPredict"][i]))

C:\Users\30133\AppData\Local\Temp/ipykernel_4164/969426254.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_x["Floor"][i] = int(np.round(df_x["FloorPredict"][i]))


In [20]:
df_x

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Embarked,Cabin,Floor,FloorPredict
0,1,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,S,NaN,1.0,1
1,2,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C,C85,5.0,5
2,3,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,S,NaN,1.0,1
3,4,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,S,C123,5.0,5
4,5,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,S,NaN,1.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,S,NaN,3.0,3
887,888,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,S,B42,6.0,3
888,889,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,S,NaN,3.0,3
889,890,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C,C148,5.0,7


In [21]:
df_x = df_x.merge(df_y, left_on="PassengerId", right_on = "PassengerId")
df_x["Floor"] = df_x["Floor"].astype('int')
df_x

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Embarked,Cabin,Floor,FloorPredict,Survived
0,1,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,S,NaN,1,1,0
1,2,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C,C85,5,5,1
2,3,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,S,NaN,1,1,1
3,4,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,S,C123,5,5,1
4,5,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,S,NaN,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,S,NaN,3,3,0
887,888,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,S,B42,6,3,1
888,889,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,S,NaN,3,3,0
889,890,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C,C148,5,7,1


In [22]:
df_x = df_x.dropna(how = 'any')
df_x = df_x.reset_index(drop=True)

In [23]:
df_y = df_x[["Survived"]]
df_x = df_x[["Pclass","Age","SibSp","Parch","Fare", "Floor"]].join(pd.get_dummies(df_x[["Sex","Embarked"]]))

In [24]:
lr_model = LogisticRegression(max_iter=5000).fit(df_x, df_y)

C:\Users\30133\anaconda3\envs\myenv\lib\site-packages\sklearn\utils\validation.py:985: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\30133\anaconda3\envs\myenv\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [25]:
df_test_x = pd.read_csv("./test.csv")
df_test_x = df_test_x[["PassengerId","Pclass","Age","SibSp","Parch","Fare", "Cabin","Sex","Embarked"]]
df_test_x

,PassengerId,Pclass,Age,SibSp,Parch,Fare,Cabin,Sex,Embarked
0,892,3,34.5,0,0,7.8292,NaN,male,Q
1,893,3,47.0,1,0,7.0000,NaN,female,S
2,894,2,62.0,0,0,9.6875,NaN,male,Q
3,895,3,27.0,0,0,8.6625,NaN,male,S
4,896,3,22.0,1,1,12.2875,NaN,female,S
...,...,...,...,...,...,...,...,...,...
413,1305,3,NaN,0,0,8.0500,NaN,male,S
414,1306,1,39.0,0,0,108.9000,C105,female,C
415,1307,3,38.5,0,0,7.2500,NaN,male,S
416,1308,3,NaN,0,0,8.0500,NaN,male,S


In [26]:
df_test_x["Floor"] = df_test_x[["Cabin"]].apply(lambda x: seperateFloor(x))
df_test_x["Pclass"] = df_test_x["Pclass"].fillna(df_test_x["Pclass"].mean())
df_test_x["Fare"] = df_test_x["Fare"].fillna(df_test_x["Fare"].mean())
df_test_x["FloorPredict"] = floor_model.predict(df_test_x[["Pclass", "Fare"]].join(pd.get_dummies(df_test_x[["Embarked"]])))
for i in range(0,df_test_x.shape[0]):
    if (pd.isna(df_test_x["Floor"][i])):
        df_test_x["Floor"][i] = int(np.round(df_test_x["FloorPredict"][i]))
df_test_x["Floor"] = df_test_x["Floor"].astype('int')
df_test_x

C:\Users\30133\AppData\Local\Temp/ipykernel_4164/1905804839.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_test_x["Floor"][i] = int(np.round(df_test_x["FloorPredict"][i]))


,PassengerId,Pclass,Age,SibSp,Parch,Fare,Cabin,Sex,Embarked,Floor,FloorPredict
0,892,3,34.5,0,0,7.8292,NaN,male,Q,1,1
1,893,3,47.0,1,0,7.0000,NaN,female,S,5,1
2,894,2,62.0,0,0,9.6875,NaN,male,Q,2,2
3,895,3,27.0,0,0,8.6625,NaN,male,S,5,1
4,896,3,22.0,1,1,12.2875,NaN,female,S,1,1
...,...,...,...,...,...,...,...,...,...,...,...
413,1305,3,NaN,0,0,8.0500,NaN,male,S,1,1
414,1306,1,39.0,0,0,108.9000,C105,female,C,5,5
415,1307,3,38.5,0,0,7.2500,NaN,male,S,1,1
416,1308,3,NaN,0,0,8.0500,NaN,male,S,1,1


In [27]:
df_test_y = df_test_x[["PassengerId"]]
df_test_x = df_test_x[["PassengerId","Pclass","Age","SibSp","Parch","Fare", "Floor"]].join(pd.get_dummies(df_test_x[["Sex","Embarked"]]))
df_test_x = df_test_x.fillna(df_test_x.mean())

In [28]:
result = lr_model.predict(df_test_x.drop("PassengerId",axis=1))

In [29]:
result

array([0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0,
       1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 1,
       1, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1,
       1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1,
       1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0,
       0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0,
       1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1,
       0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1,
       1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1,
       0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0,
       1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1,
       0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1,
       0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0,
       0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0,

In [30]:
def classification(x) :
    if x > 0.5 :
        return 1
    else :
        return 0

In [31]:
result = pd.DataFrame(result, columns = ["Survived"])

In [32]:
result["Survived"] = result["Survived"].apply(lambda x: classification(x))

In [33]:
result = result.join(df_test_y)
result = result[["PassengerId","Survived"]]
result

,PassengerId,Survived
0,892,0
1,893,1
2,894,0
3,895,0
4,896,1
...,...,...
413,1305,0
414,1306,1
415,1307,0
416,1308,0


In [34]:
result.to_csv('./submission.csv',index=False)